# SyftBox SDK Development Testing

This notebook is for testing and validating the SyftBox SDK during development.

In [1]:
%%capture output --no-stderr

# Install the SDK in development mode (run this once)
!pip install nest_asyncio
!pip install -e .

In [2]:
# Import the SDK
import syft_nsai_sdk as sdk
import nest_asyncio
import httpx
import logging

# 
nest_asyncio.apply()

In [3]:
from syft_nsai_sdk.utils.logger import get_logger
from syft_nsai_sdk import SyftBoxClient
from IPython.display import display, HTML, Markdown

logger = get_logger()
logger.success("SyftBox SDK imported successfully")

# Turn off debug logging for your SDK
logging.getLogger('syft_nsai_sdk').setLevel(logging.INFO)

2025-08-27 17:38:30 - syft_nsai_sdk - SUCCESS - SyftBox SDK imported successfully


In [4]:
client = SyftBoxClient()
logger.info(f"Syftbox client created: {client}")

2025-08-27 17:38:32 - syft_nsai_sdk.main - INFO - Accounting configured for callis@openmined.org
2025-08-27 17:38:32 - syft_nsai_sdk.main - INFO - SyftBoxClient initialized for guest@syft.org
2025-08-27 17:38:32 - syft_nsai_sdk - INFO - Syftbox client created: <syft_nsai_sdk.main.SyftBoxClient object at 0x105f8d430>


In [5]:
# STEP 2: Discover models explicitly 
models = client.discover_models(service_type="chat")
print(f"Found {len(models)} models")

2025-08-27 17:38:34 - syft_nsai_sdk.discovery.scanner - INFO - Found 10 models across 256 datasites
2025-08-27 17:38:34 - syft_nsai_sdk.discovery.scanner - INFO - Cached 10 models from 8 owners
2025-08-27 17:38:34 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for keelan-test-3: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-27 17:38:34 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for carl-model: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-27 17:38:34 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for carl-free: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-27 17:38:34 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for claude-sonnet-3.5: AUTHENTICATION_ERROR: No accounting service URL configured
2025-08-27 17:38:34 - syft_nsai_sdk.services.health - WARNING - Unexpected error in health check for deepseek

Found 6 models


In [6]:
print(f"{'Name':<20} {'Owner':<25} {'Tags':<25} {'Services':<25}")
print("-" * 100)
for model in models:
    enabled_services = [s.type.value for s in model.services if s.enabled]
    tags_str = ", ".join(model.tags[:2])
    services_str = ", ".join(enabled_services)
    print(f"{model.name:<20} {model.owner:<25} {tags_str:<25} {services_str:<25}")

Name                 Owner                     Tags                      Services                 
----------------------------------------------------------------------------------------------------
keelan-test-3        kj@kj.dev                 test                      chat, search             
carl-model           callis@openmined.org      Chat, Search              chat, search             
carl-free            callis@openmined.org      chat, search              chat, search             
claude-sonnet-3.5    aggregator@openmined.org  claude, anthropic         chat, search             
deepseek-v3          aggregator@openmined.org  deepseek, opensource      chat                     
test                 irina@openmined.org       test                      chat, search             


In [7]:
# Check env vars are loaded
import os

email = os.getenv('SYFTBOX_ACCOUNTING_EMAIL')
password = os.getenv('SYFTBOX_ACCOUNTING_PASSWORD')
service_url = os.getenv('SYFTBOX_ACCOUNTING_URL')
print(f"SYFTBOX_ACCOUNTING_EMAIL: {email}")
# print(f"SYFTBOX_ACCOUNTING_PASSWORD: {password}")
# print(f"SYFTBOX_ACCOUNTING_URL: {service_url}")
# print(f"SYFTBOX_ACCOUNTING_EMAIL: {bool(email)}")
print(f"SYFTBOX_ACCOUNTING_PASSWORD: {bool(password)}")
print(f"SYFTBOX_ACCOUNTING_URL: {bool(service_url)}")

SYFTBOX_ACCOUNTING_EMAIL: callis@openmined.org
SYFTBOX_ACCOUNTING_PASSWORD: True
SYFTBOX_ACCOUNTING_URL: True


In [8]:
print(await client.get_account_info())

{'email': 'callis@openmined.org', 'balance': 184.9, 'currency': 'USD'}


In [9]:
# model = client.find_model("claude-sonnet-3.5", "aggregator@openmined.org")
# model

In [10]:
# Create conversation (enhanced with owner support)
conversation = client.create_conversation(
    model_name="claude-sonnet-3.5",
    owner="aggregator@openmined.org",
)
# Each message remembers full context
response1 = await conversation.send_message("What is SyftBox?")

2025-08-27 17:38:44 - syft_nsai_sdk.main - INFO - Discovered 1 models (health_check=False)
2025-08-27 17:38:44 - syft_nsai_sdk.services.chat - INFO - Building RPC payload for chat request: ChatRequest(messages=[ChatMessage(role='user', content='What is SyftBox?', name=None)], model='claude-sonnet-3.5', options=None, user_email='guest@syft.org', transaction_token=None)


2025-08-27 17:40:47 - syft_nsai_sdk.services.chat - ERROR - Chat request failed for model claude-sonnet-3.5: RPC_ERROR: Polling timed out after 30/30 attempts


✓ Response received



PollingTimeoutError: RPC_ERROR: Polling timed out after 30/30 attempts

In [11]:
print(f"Response:\n{response1}")

Response:
ChatResponse(id='7d8285a1-577a-459d-b7ab-fbc7d85bbdc8', model='claude-sonnet-3.5', message=ChatMessage(role='assistant', content="{'detail': 'Chat completion failed'}", name=None), usage=ChatUsage(prompt_tokens=0, completion_tokens=0, total_tokens=0), cost=None, provider_info=None)


In [ ]:
# second conversation
response2 = await conversation.send_message("How does it work?")  # Remembers first question

In [ ]:
print(f"Response:\n{response2}")

In [ ]:
# third conversation
response3 = await conversation.send_message("Give an example")    # Remembers both previous

In [ ]:
print(f"Response:\n{response3}")